In [1022]:
from jsonobject import *
import numpy as np 
import random
import math
import matplotlib as mpl
import matplotlib.pyplot as plt

In [1023]:
random.seed(2021)

# Classes

In [1024]:
# Convolution
class Convolution(JsonObject):
    kernel = IntegerProperty()
    padding = StringProperty()
    stride = IntegerProperty()
    nb_filter = IntegerProperty()
    fct_activation = StringProperty()

In [1025]:
# Input 
class InputLayer(JsonObject):
    shape = ListProperty(int)

In [1026]:
# Pooling Avg/Max
class Pooling(JsonObject):
    op = StringProperty()
    kernel = IntegerProperty(default=2)
    padding = StringProperty(default="valid")
    stride = IntegerProperty(default=None)

In [1027]:
# Class Flatten
class Flatten(JsonObject):
    pass

In [1028]:
# Dense --> Fully connected layer
class Dense(JsonObject):
    nb_neurones =  IntegerProperty()
    fct_activation = StringProperty()

In [1029]:
# Batchnorm
class BatchNormalisation(JsonObject):
    epsilon = FloatProperty()

In [1030]:
# Global pooling 
class GlobalPooling(JsonObject):
    op = StringProperty()

In [1031]:
# classe for the different possible merge add/concatenate
class Merge(JsonObject):
    m_type = StringProperty()
    sub_SM_1 = ListProperty(JsonObject)
    sub_SM_2 = ListProperty(JsonObject)

In [1032]:
# class dropout
class Dropout(JsonObject):
    dropout_rate = FloatProperty()   

In [1033]:
# class that represente when the way of merge is empty
class Empty(JsonObject):
    pass

# Value

In [1034]:
# Table of hyperparameter value
kernel_value = [1, 2, 3, 4, 5, 6, 7]
stride_value = [1, 2, 3, 4, 5, 6, 7]
padding_value = ["valid", "same"]
fct_activation_value = ["tanh", "relu", "selu"]

epsilon_value = [0, 1.001e-5, 0.001, 1.1e-5, 1.1e-7]
dropout_value = [0, .1, .4, .5, .8, .01, .001]

nb_layer_value = [1,2,3,4,5,6]

nb_class = 10 # nb_class
op_value = ['avg',"max"]

m_type_value = ["add", "concatenate"]

classification = [Flatten, GlobalPooling]

merge_type = ["full", "empty"]

# Functions

In [1035]:
def equals_sm1_sm2(ssm1, ssm2):
    try:
        i = 0
        while (i<len(ssm1)):
            if (ssm1[i] != ssm2[i]):
                return False
            i+=1
        return True
    except: return False

In [1036]:
def stop(size_archi, x):
    prob = (math.log(2*size_archi)*x)-(2*x)
    if ( prob < random.randrange(101)):
        return True
    else : return False

In [1037]:
# Create Json file
def create_json_file(archi, file_name):
    directory = 'architecture_json/'
    
    # reset file
    archi_file = open(directory+file_name, "w")
    archi_file.close()
    
    # create file
    
    # open file
    archi_file = open(directory+file_name, "a")
    
    archi_file.write("""[
    """)
    
    archi_size = len(archi)
    i = 0
    for l in archi:
        if(isinstance(l, Merge)):
            str_layer = """{
        'class':'"""
            str_layer += l.__class__.__name__
            str_layer +="""',\n\t\t'parameters':{"""
            str_layer += "'m_type': '" + str(l.m_type) + "',"
            str_layer += "\n\t\t\t'sub_SM_1': [\n\t\t"
            j = 0
            for sl in l.sub_SM_1:
                str_layer += """\t\t\t{
                    'class':'"""
                str_layer += sl.__class__.__name__
                str_layer +="""',\n\t\t\t\t\t'parameters':"""
                str_layer += str(sl.to_json())
                str_layer += """\n\t\t\t\t}"""
                
                if(j < len(l.sub_SM_1)-1):
                    str_layer += ""","""
                    j+=1
            str_layer += "],"        
            str_layer += "\n\t\t\t'sub_SM_2': [\n"
            j = 0
            for sl in l.sub_SM_2:
                str_layer += """\t\t\t\t{
                    'class':'"""
                str_layer += sl.__class__.__name__
                str_layer +="""',\n\t\t\t\t\t'parameters':"""
                str_layer += str(sl.to_json())
                str_layer += """\n\t\t\t\t}"""
                
                if(j < len(l.sub_SM_2)-1):
                    str_layer += ""","""
                    j+=1
            
            str_layer += "]" 
            str_layer += """}\n\t\t}"""
            if( i < archi_size-1):
                str_layer += ""","""
                i+=1
            str_layer = str_layer.replace("'","\"")
        else: 
            str_layer = """{
        'class':'"""
            str_layer += l.__class__.__name__
            str_layer +="""',\n\t\t'parameters':"""
            str_layer += str(l.to_json())
            str_layer += """\n\t}"""
            if(i < archi_size-1):
                str_layer += ""","""
                i+=1
            str_layer = str_layer.replace("'","\"")
        
        archi_file.write(str_layer)
        
    archi_file.write("""\n]""")
    archi_file.close()

In [1038]:
#instantate the layer to add
def construct_layer(layer, nb_filter_value, kernel=None, stride=None, padding=None):
    
    if(kernel == None):    
        # Kernel <= output/2
        if(output_size == 2):
            kernel_value_filtered = [1,2]
        elif(output_size == 1):
            kernel_value_filtered = [1]
        else: 
            kernel_value_filtered = [value for value in kernel_value if value <= math.ceil(output_size/2)]

        kernel=kernel_value_filtered[random.randrange(len(kernel_value_filtered))]
    
    if(stride == None):
        # kernel >= stride
        stride_value_filtered = [value for value in stride_value if value <= kernel]
        stride = stride_value_filtered[random.randrange(len(stride_value_filtered))]
    
    if(padding == None):
        padding=padding_value[random.randrange(2)]
    # if layer is Convolution
    if(layer == Convolution):
        add_layer = Convolution(
            kernel=kernel, 
            padding=padding, 
            stride=stride, 
            nb_filter= nb_filter_value,
            fct_activation=fct_activation_value[random.randrange(3)]
        )
    
    # if is pooling
    elif(layer == Pooling):
        add_layer = Pooling(
            op = op_value[random.randrange(2)],
            kernel=kernel, 
            padding=padding, 
            stride=stride
        )
    #if is BN
    elif(layer == BatchNormalisation):
        add_layer = BatchNormalisation(epsilon = epsilon_value[random.randrange(len(epsilon_value))])
        
    #if dropout
    elif(layer == Dropout):
        add_layer = Dropout(dropout_rate = dropout_value[random.randrange(len(dropout_value))])
    
    # if Empty
    elif(layer == Empty):
        add_layer = Empty()
            
    else: 
        print("construct layer error")
    
    return add_layer

In [1039]:
# add layer to architecture 

def addLayer(archi, add_layer, output_size):
    
    input_size = output_size
    feature_extra = archi[1:]
    
    if( (add_layer == Dropout) | (add_layer == BatchNormalisation)):
        archi.append(add_layer)
        return 1, archi, output_size
    
    # add layer if the architecture is empty
    elif(feature_extra == []):
        archi.append(add_layer)
        return 1, archi, calcul_output(input_size, add_layer)
    
    else :
        
        # if we couldn't reduce more
        #if(input_size == 1):
        #    return 0, archi, input_size
        
        # if the output size got more than 1 we can add new layer
        if(output_size >= 1): 
            output_size = calcul_output(input_size, add_layer)
            
            # if output size got negate is that the layer we want to add is wrong
            if(output_size < 1 ):
                return -1, archi, input_size
            # if output size is bigger than 0 we can add new layer and continue
            elif(output_size > 0):
                archi.append(add_layer)
                return 1, archi, output_size
            # this should not append
            else:
                return "Somethink wrong"
    # this should not append
    return "Error"

In [1040]:
# compute the output 
# valid vs same
def calcul_output(input_size, l):
    output_size = 0
    
    if( (isinstance(l, Empty)) | (isinstance(l, BatchNormalisation)) | (isinstance(l, Dropout))):
        return input_size
    if(l.padding == "valid"):
        kernel = l.kernel
        stride = l.stride
        while(input_size>=kernel):
            input_size -= stride
            output_size += 1   
    else:
        stride = l.stride
        if(input_size%stride == 0):
            output_size = int(input_size/stride)
        else:
            output_size = int(input_size/stride)+1
    return output_size

In [1041]:
def build_empty_merge(sm, output, nb_filter_value):
    nb_layer = random.randrange(len(nb_layer_value))
    for i in range(0, nb_layer):
        
        if(len(sm) > 2):
            layer = getSub_SM_Layer(last_layer=sm[len(sm)-1], prev_last_layer=sm[len(sm)-2], tag="empty")
        elif(len(sm)>1):
            layer = getSub_SM_Layer(last_layer=sm[len(sm)-1], tag="empty")
        else:
            layer = getSub_SM_Layer(tag="empty")
        
        if(output == 2):
            kernel_value_filtered = [1,2]
        elif(output == 1):
            kernel_value_filtered = [1]
        else: 
            kernel_value_filtered = [value for value in kernel_value if value <= math.ceil(output/2)]

        kernel=kernel_value_filtered[random.randrange(len(kernel_value_filtered))]
        
        
        add_layer = construct_layer(layer, nb_filter_value, padding="same", stride=1, kernel=kernel)
        _, sm, output = addLayer(sm, add_layer, output)
        
    return sm, output

In [1042]:
# build the merge part 
def build_merge(archi, output_size, nb_filter_value):
    sub_sm1 = list()
    sub_sm2 = list()
    m_type = m_type_value[random.randrange(2)]
    
    #m_type = merge_type[random.randrange(2)]
    m_type = merge_type[1]
    
    if(m_type == "empty"):
        sub_sm1, output_size_sm1 = build_empty_merge(sub_sm1, output_size, nb_filter_value)
        add_layer = construct_layer(Empty, nb_filter_value)
        _, sub_sm2, output_size_sm2 = addLayer(sub_sm2, add_layer, output_size)
    
    else: 
        print("non empty")
    
    m = Merge(m_type=m_type,
             sub_SM_1=sub_sm1,
             sub_SM_2=sub_sm2)
    
    if(equals_sm1_sm2(sub_sm1, sub_sm2)):
        #build_merge(archi, output_size)
        print("equals")
        return archi, output_size
    else: 
        archi.append(m)
        return archi , output_size

In [1043]:
# Convolution, Pooling, BatchNormalisation,  Dropout, Merge, Empty
# Return list of layer compared to last layer for the sub state machine
def getSub_SM_Layer(last_layer=None, prev_last_layer= None, tag="init"):
    list_layer = list()
    if(tag == "init"):
        list_layer = [Convolution, BatchNormalisation] # Merge
    
    elif(tag == "empty"):
        if(isinstance(last_layer, Convolution)):
                list_layer = [Convolution, BatchNormalisation, Dropout]
        elif(isinstance(last_layer, BatchNormalisation)):
            if(isinstance(prev_last_layer, Convolution)):
                list_layer = [Convolution, Dropout]
            else: list_layer = [Convolution]
        else: list_layer = [Convolution]
    else: 
        if(isinstance(last_layer, Convolution)):
            list_layer = [Convolution, Pooling, BatchNormalisation, Dropout]
        elif(isinstance(last_layer, Pooling)):
            list_layer = [Convolution]
        elif(isinstance(last_layer, BatchNormalisation)):
            if(isinstance(prev_last_layer, Convolution)):
                list_layer = [Convolution, Pooling, Dropout]
            else: list_layer = [Convolution]
        elif(isinstance(last_layer, Dropout)):
            list_layer = [Pooling]
    return list_layer[random.randrange(len(list_layer))]

In [1044]:
# Convolution, Pooling, BatchNormalisation, Dropout, Merge
# Return list of layer compared to last layer
def getlayer(last_layer, prev_last_layer):
    list_layer = list()
    if(isinstance(last_layer, InputLayer)):
        list_layer = [Convolution, BatchNormalisation, Merge]
    elif(isinstance(last_layer, Convolution)):
        list_layer = [Convolution, Pooling, BatchNormalisation,  Dropout, Merge]
    elif(isinstance(last_layer, Pooling)):
        list_layer = [Convolution, Merge]
    elif(isinstance(last_layer, BatchNormalisation)):
        if(isinstance(prev_last_layer, Convolution)):
            list_layer = [Convolution, Merge, Pooling, Dropout]
        else: list_layer = [Convolution]
    elif(isinstance(last_layer, Dropout)):
        list_layer = [Pooling]
    elif(isinstance(last_layer, Merge)):
        list_layer = [Convolution, BatchNormalisation, Merge]
    else: 
        print("getLayer error")
        print(last_layer)
    return list_layer[random.randrange(len(list_layer))]

# Creation Architecture

In [1045]:
architecture = list()

nb_filter_value = 1 # init nb feature map
output_size = 28

# add input Layer
architecture.append(InputLayer(shape=[28,28,1]))

# add extraction feature (succession of Pooling/convolution)
# Pooling can't be follow by a Pooling
pooling = True
code = 1 # we can add new layer
j = 6
last_layer = architecture[len(architecture)-1]

while( ((isinstance(last_layer, Dropout)) | (isinstance(last_layer, BatchNormalisation))) | 
      (output_size == 28) | 
      ((stop(len(architecture),7)) & (code == 1))):
    
    layer = getlayer(architecture[len(architecture)-1], architecture[len(architecture)-2])
    
    
    if (layer == Merge):
        architecture, output_size = build_merge(architecture, output_size, nb_filter_value)
    else:
        
        if(layer == Convolution):
            nb_filter_value = nb_filter_value*j
            j=2
        add_layer = construct_layer(layer, nb_filter_value)

        code, architecture, output_size = addLayer(architecture, add_layer, output_size)
        last_layer = architecture[len(architecture)-1]


try:
    nb_filter_value = architecture[len(architecture)-1]['nb_filter']
except:
    nb_filter_value = int(nb_filter_value/2)
    

print("output size : " + str(output_size))
print("nb_filter : " + str(nb_filter_value))
print("code: " + str(code))

clf_layer = classification[random.randrange(2)]
print(clf_layer)
if clf_layer == Flatten:
    
    # add flatten layer
    architecture.append(Flatten())
    
    
    # compute parameters
    param = output_size*output_size*(nb_filter_value)
    print('param : ' + str(param))


    ## init values
    nb=0
    if (param >= 1000):
        nb_neurones = int(param*20/100)
        print("=====")
        print("iteration" + str(nb))
        print("nb_neurone : " + str(nb_neurones))
        dense = Dense(
            nb_neurones = nb_neurones,
            fct_activation = fct_activation_value[random.randrange(3)]
        )
        architecture.append(dense)
        nb+=1
        nb_neurones = int(nb_neurones*20/100)

    else:
        pourcent = random.uniform(10,90)
        nb_neurones = int(param*pourcent/100)

    while(nb_neurones >= 1000):
        print("=====")
        print("iteration" + str(nb))
        print("nb_neurone : " + str(nb_neurones))
        dense = Dense(
            nb_neurones = nb_neurones,
            fct_activation = fct_activation_value[random.randrange(3)]
        )
        architecture.append(dense)
        nb+=1
        nb_neurones = int(nb_neurones*20/100)


    ## add dense Layers
    while(nb_neurones > nb_class*10):
        print("=====")
        print("iteration" + str(nb))
        print("nb_neurone : " + str(nb_neurones))

        dense = Dense(
            nb_neurones = nb_neurones,
            fct_activation = fct_activation_value[random.randrange(3)]
        )
        architecture.append(dense)


        #incrementation
        pourcent = random.uniform(10,90)
        nb+=1
        nb_neurones = int(nb_neurones*pourcent/100)

elif (clf_layer == GlobalPooling):
    architecture.append(GlobalPooling(op=op_value[random.randrange(2)]))
else: print("clf error")


last_dense = Dense(
    nb_neurones = nb_class,
    fct_activation="softmax"           
)

architecture.append(last_dense)            
        
        
print(architecture)


# create file
file_name = "archi_V4_test.json"
print(file_name)
print("")
print("")
create_json_file(architecture, file_name)


output size : 2
nb_filter : 24
code: 1
<class '__main__.GlobalPooling'>
[InputLayer(shape=[28, 28, 1]), Merge(m_type='empty', sub_SM_1=[Convolution(fct_activation='relu', kernel=4, nb_filter=1, padding='same', stride=1)], sub_SM_2=[Empty()]), BatchNormalisation(epsilon=1.001e-05), Convolution(fct_activation='tanh', kernel=6, nb_filter=6, padding='same', stride=6), Dropout(dropout_rate=0.01), Pooling(kernel=3, op='avg', padding='same', stride=2), Convolution(fct_activation='tanh', kernel=2, nb_filter=12, padding='same', stride=2), Merge(m_type='empty', sub_SM_1=[Convolution(fct_activation='tanh', kernel=2, nb_filter=12, padding='same', stride=1), Convolution(fct_activation='tanh', kernel=1, nb_filter=12, padding='same', stride=1), Dropout(dropout_rate=0.1)], sub_SM_2=[Empty()]), Merge(m_type='empty', sub_SM_1=[Convolution(fct_activation='relu', kernel=1, nb_filter=12, padding='same', stride=1), Convolution(fct_activation='tanh', kernel=1, nb_filter=12, padding='same', stride=1), Dropout